<a href="https://www.kaggle.com/code/iman1733/strokepredictiondt?scriptVersionId=104556673" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Imports
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf

In [2]:
# Load in the dataset
dataset = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

dataset = dataset.sample(frac=1)

# Get rid of unnecessary columns
dataset = dataset.drop(columns=['id'])

# Fill any missing values with the mean of the column
mean_bmi = dataset['bmi'].mean()
dataset['bmi'] = dataset['bmi'].fillna(mean_bmi)

dataset.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
4545,Male,16.0,0,0,No,children,Urban,135.82,35.1,never smoked,0
569,Female,32.0,0,0,No,Self-employed,Urban,65.30,40.4,never smoked,0
4494,Male,54.0,0,0,No,Private,Rural,106.52,27.4,formerly smoked,0
4107,Female,59.0,0,0,Yes,Self-employed,Urban,77.60,23.4,Unknown,0
1896,Female,25.0,0,0,No,Govt_job,Rural,74.11,34.1,smokes,0


In [3]:
# Convert the data into a more readable format for the model
#dataset.replace(to_replace=["Female","Male"], value=[0,1], inplace=True)
dataset.replace(to_replace=["Rural","Urban"], value=[0,1], inplace=True)
dataset.replace(to_replace=["No","Yes"], value=[0,1], inplace=True)

dataset = pd.get_dummies(dataset, columns=['work_type', 'smoking_status','gender'])

dataset.head()

,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,gender_Female,gender_Male,gender_Other
4545,16.0,0,0,0,1,135.82,35.1,0,0,0,0,0,1,0,0,1,0,0,1,0
569,32.0,0,0,0,1,65.30,40.4,0,0,0,0,1,0,0,0,1,0,1,0,0
4494,54.0,0,0,0,0,106.52,27.4,0,0,0,1,0,0,0,1,0,0,0,1,0
4107,59.0,0,0,1,1,77.60,23.4,0,0,0,0,1,0,1,0,0,0,1,0,0
1896,25.0,0,0,0,0,74.11,34.1,0,1,0,0,0,0,0,0,0,1,1,0,0


In [4]:
# Split the data into training data and testing data using an 80:20 split
train_size = round(0.8 * len(dataset))
test_size = round(0.2 * len(dataset))

train_features = dataset[:train_size]
test_features = dataset[train_size:]

# Taking off the stroke column as this is what we want to predict
train_labels = train_features.pop('stroke')
test_labels = test_features.pop('stroke')

In [5]:
train_features.head()

,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,gender_Female,gender_Male,gender_Other
4545,16.0,0,0,0,1,135.82,35.1,0,0,0,0,1,0,0,1,0,0,1,0
569,32.0,0,0,0,1,65.30,40.4,0,0,0,1,0,0,0,1,0,1,0,0
4494,54.0,0,0,0,0,106.52,27.4,0,0,1,0,0,0,1,0,0,0,1,0
4107,59.0,0,0,1,1,77.60,23.4,0,0,0,1,0,1,0,0,0,1,0,0
1896,25.0,0,0,0,0,74.11,34.1,1,0,0,0,0,0,0,0,1,1,0,0


In [6]:
# Make the model
model = DecisionTreeRegressor(random_state=1)
model.fit(train_features, train_labels)

DecisionTreeRegressor(random_state=1)

In [7]:
# Making predictions
predictions = model.predict(test_features)
correct = 0

for i in range(len(test_labels)):
    if int(predictions[i]) == test_labels.iloc[i]:
        correct += 1

print("Number of correct values: ", correct, " out of ", len(test_labels))
print("Accuracy: ", (correct / len(test_labels)))

Number of correct values:  940  out of  1022
Accuracy:  0.9197651663405088
